In [1]:
import pandas as pd
import numpy as np
from numpy import mean
from numpy import std
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.feature_selection import RFECV,SelectKBest,mutual_info_classif,SelectFromModel
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV,LeaveOneOut,StratifiedKFold,RepeatedKFold,RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,BaggingClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score,roc_auc_score,roc_curve,auc,confusion_matrix
from mat4py import loadmat
import matplotlib.pyplot as plt 
import math
import random
random.seed(24)
import warnings
warnings.filterwarnings("ignore")
from sklearn.feature_selection import VarianceThreshold

In [3]:
# Case-based difficuty score
case_feat_normal = pd.read_csv('/Users/jessy/Desktop/笔记本/Radiomics Data Analysis/diffScore w_out 10 dep Cn/case_feat_diffScore/RA_norm_feat.csv')
case_feat_normal = pd.DataFrame(case_feat_normal).drop("Unnamed: 0",axis=1)
print('Whether there is missing value in normal mat: {}'.format(case_feat_normal.isnull().values.any())) 
case_feat_cancer = pd.read_csv('/Users/jessy/Desktop/笔记本/Radiomics Data Analysis/diffScore w_out 10 dep Cn/case_feat_case_cancer_diffScore/RA_norm_feat.csv')
case_feat_cancer = pd.DataFrame(case_feat_cancer).drop("Unnamed: 0",axis=1)
print('Whether there is missing value in cancer mat: {}'.format(case_feat_cancer.isnull().values.any())) 

Whether there is missing value in normal mat: False
Whether there is missing value in cancer mat: False


In [4]:
CC_Normal = case_feat_normal.loc[case_feat_normal['View']=="'CC'",]
MLO_Normal = case_feat_normal.loc[case_feat_normal['View']=="'MLO'",]
left_CC = CC_Normal.loc[CC_Normal["Side"]=="L",]
right_CC =CC_Normal.loc[CC_Normal["Side"]=="R",]
left_MLO = MLO_Normal.loc[MLO_Normal["Side"]=="L",]
right_MLO = MLO_Normal.loc[MLO_Normal["Side"]=="R",]
lesion_side = pd.read_csv("/Users/jessy/Desktop/笔记本/Radiomics Data Analysis/diffScore w_out 10 dep Cn/lesion_side.csv")
case_feat_cancer_ = case_feat_cancer.merge(lesion_side, how='left',on = "CaseName").drop("Unnamed: 0",axis=1)
CC_Cancer = case_feat_cancer_.loc[case_feat_cancer_["Side"]==case_feat_cancer_["LesionSide"] ,].loc[case_feat_cancer_["View"]=="'CC'",]
MLO_Cancer = case_feat_cancer_.loc[case_feat_cancer_["Side"]==case_feat_cancer_["LesionSide"] ,].loc[case_feat_cancer_["View"]=="'MLO'",]

CC_Cancer.drop(17, axis=0, inplace=True)
MLO_Cancer.drop(23, axis=0, inplace=True)
print('The shape of CC_Normal is {}'.format(CC_Normal.shape))
print('The shape of CC_CancerLesion is {}'.format(CC_Cancer.shape))
print('The shape of MLO_Normal is {}'.format(MLO_Normal.shape))
print('The shape of MLO_CancerLesion is {}'.format(MLO_Cancer.shape))

The shape of CC_Normal is (80, 210)
The shape of CC_CancerLesion is (20, 212)
The shape of MLO_Normal is (80, 210)
The shape of MLO_CancerLesion is (20, 212)


# algorithms to use: Logistic regression
def self_LR_pipe(X,y,pipe, param):
    random.seed(24)
    cv_outer = LeaveOneOut()
    y_true,y_pred = list(),list()
    for train_ix, test_ix in cv_outer.split(X):
        X_train_, X_test = X[train_ix, :], X[test_ix, :]
        y_train_, y_test = y[train_ix], y[test_ix]

        
        # inner loop for feature selection and hyperparameter tuning 
        pipe_lr = pipe
        param_grid = param
        
        cv_inner = StratifiedKFold(n_splits=3, shuffle=True, random_state=24)
        result = GridSearchCV(pipe_lr, param_grid, cv = cv_inner, n_jobs= -1,scoring = 'roc_auc',refit=True).fit(X_train_, y_train_.ravel())
        best_param = result.best_estimator_
        # evaluate model on the hold out validation set
        yhat_proba = list(best_param.predict_proba(X_test)[:,1])
        y_pred.append(yhat_proba)
        y_true.append(list(y_test[0]))
        # report progress
        print('>est=%.3f, cfg=%s' % (result.best_score_, result.best_params_))
        print('Selected features are: {}'.format(np.where(result.best_estimator_.named_steps['features'].get_support())[0]))
    # Calculate roc_auc on the hold out dataset
    print('auc: %.3f' % roc_auc_score(y_true, y_pred))  
    


In [4]:
# algorithms to use: Random Forest
def self_rf_pipe(X,y,pipe, param):
    features = {}
    row = 0
    random.seed(4)
    cv_outer = LeaveOneOut()
    y_true,y_pred,Predicted_class  = list(),list(),list()
    for train_ix, test_ix in cv_outer.split(X):
        row+=1
        X_train, X_test = X[train_ix, :], X[test_ix, :]
        y_train, y_test = y[train_ix], y[test_ix]

        ## inner loop for feature selection and hyperparameter tuning 

        cv_inner = RepeatedStratifiedKFold(n_splits=3, n_repeats=3, random_state=2)
        result = GridSearchCV(pipe, param_grid=param, cv = cv_inner, scoring = 'roc_auc',n_jobs = -1,refit=True).fit(X_train, y_train.ravel())
        best_param = result.best_estimator_
        # evaluate model on the hold out evaluation dataset
        
        yhat_proba = best_param.predict_proba(X_test)[:,1]# reture the probability of predicting '1'
        y_pred.append(yhat_proba[0])
        Predicted_class.append(best_param.predict(X_test))
        y_true.append(y_test[0])
        # report progress
        print('>est=%.3f, cfg=%s' % (result.best_score_, result.best_params_))
        features[row] = np.where(result.best_estimator_.named_steps['features'].get_support())[0]

    
    Dict = {}
    for key in sorted(features):
        for num in features[key]:
            if num not in Dict:
                Dict[num] = 1
            else:
                Dict[num]+=1
    
    KEY = []

    for key in sorted(Dict):
        print(key,':',Dict[key])
        if Dict[key] >= len(X)/2:
            KEY.append(key)
    print(KEY) 

    # Calculate roc_auc on the hold out dataset
    AUC_score = roc_auc_score(y_true, y_pred)
    Accuracy = accuracy_score(y_true,Predicted_class)
    print('auc: %.3f' % AUC_score)
    print("Accuracy: ", Accuracy)
  
    print(y_true)
    print(y_pred)

    
    cm1 = confusion_matrix(y_true,Predicted_class)
    sensitivity1 = cm1[0,0]/(cm1[0,0]+cm1[0,1])
    print('Sensitivity : ', sensitivity1 )

    specificity1 = cm1[1,1]/(cm1[1,0]+cm1[1,1])
    print('Specificity : ', specificity1)
    return Dict,AUC_score,Accuracy,sensitivity1,specificity1

In [5]:
# ALGORITHM: to evaluate the entire pipeline
def pipeline_evaluate(X,y,pipe, param):
    random.seed(24)
    cv_outer = LeaveOneOut()
    y_true,y_pred, Predicted_class  = list(),list(),list()
    for train_ix, test_ix in cv_outer.split(X):
        X_train, X_test = X[train_ix, :], X[test_ix, :]
        y_train, y_test = y[train_ix], y[test_ix]

        ## inner loop for feature selection and hyperparameter tuning 
        cv_inner = RepeatedStratifiedKFold(n_splits=3, n_repeats=3, random_state=2)
        result = GridSearchCV(pipe, param_grid=param, cv = cv_inner, scoring = 'roc_auc',n_jobs = -1,refit=True).fit(X_train, y_train.ravel())
        best_param = result.best_estimator_
        # evaluate model on the hold out evaluation dataset
        yhat_proba = best_param.predict_proba(X_test)[:,1]# reture the probability of predicting '1'
        y_pred.append(yhat_proba[0])
        Predicted_class.append(best_param.predict(X_test))
        y_true.append(y_test[0])
        # report progress
        print('>est=%.3f, cfg=%s' % (result.best_score_, result.best_params_))
    return y_pred,Predicted_class, y_true


# Normal Cases

In [6]:
Side_View = {}
Side_View = {'left_CC':left_CC,
             'right_CC':right_CC,
             'left_MLO':left_MLO,
             'right_MLO':right_MLO}
rads = ['diffScore.CN','diffScore.AU']

## Chinese Normal

# logistic Regression 02 (log, pca + mutual_info, LR)
rad = rads[0]
bins = Bins[0]
for key,value in Side_View.items():
    Normal_sort = value.sort_values(by = rad)
    Normal_sort_copy = Normal_sort.copy()
    Normal_sort_copy['percentile']=pd.cut(Normal_sort_copy[rad],bins, labels=[1,3,0])
    Normal_sort_drop = pd.concat([Normal_sort_copy[Normal_sort_copy['percentile']==1],Normal_sort_copy[Normal_sort_copy['percentile']==0]],axis = 0)
    
    print('ThIS IS {} SIDE VIEW'.format(key),'\n\n')
    X = Normal_sort_drop.loc[:,0:202]
    y = Normal_sort_drop['percentile'].to_numpy().reshape(-1,1)

    # prepare data (log transform)
    from scipy.stats import skew
    skewness = X.apply(lambda x: skew(x))
    skewed_feats = skewness[skewness > 0.75]
    skewed_feats = skewed_feats.index
    minimum = X[skewed_feats].min(axis = 1).min()
    skewed_feats = np.log1p(X[skewed_feats]+ abs(minimum))
    X = X.to_numpy()

    pca = PCA(n_components = 0.9)
    selection = SelectKBest(mutual_info_classif,k=10)
    combined_features = FeatureUnion([("pca", pca), ("univ_select", selection)])

    pipe_lr2 = Pipeline([
        ('scaler',MinMaxScaler()),
        ('features',combined_features),
        ('lr',LogisticRegression(solver='liblinear',random_state=42))])
    param_lr2 = {'features__univ_select__k':[5,10,20,30],
                'lr__penalty':['l1', 'l2'],
                'lr__C':[1000,100,10,1.0,0.1,0.01,0.001]} 

    print('Below is the results from the LR02:')


    cv_outer = LeaveOneOut()
    y_true,y_pred = list(),list()
    for train_ix, test_ix in cv_outer.split(X):
        X_train_, X_test = X[train_ix, :], X[test_ix, :]
        y_train_, y_test = y[train_ix], y[test_ix]

        # inner loop for feature selection and hyperparameter tuning 
        cv_inner = StratifiedKFold(n_splits=3, shuffle=True, random_state=24)
        result = GridSearchCV(pipe_lr2, param_lr2, cv = cv_inner, n_jobs= -1,scoring = 'roc_auc',refit=True).fit(X_train_, y_train_.ravel())
        best_param = result.best_estimator_
        # evaluate model on the hold out validation set
        yhat_proba = list(best_param.predict_proba(X_test)[:,1])
        y_pred.append(yhat_proba)
        y_true.append(list(y_test[0]))
        # report progress
        print('>est=%.3f, cfg=%s' % (result.best_score_, result.best_params_))
       # Calculate roc_auc on the hold out dataset
    print('auc: %.3f' % roc_auc_score(y_true, y_pred))    

# For Chinese Rads Logistic regression 1 3 
print('This is for Chinese Radiologists: \n')
rad = rads[0]
bins = Bins[0]
for key,value in Side_View.items():
    Normal_sort = value.sort_values(by = rad)
    Normal_sort_copy = Normal_sort.copy()
    Normal_sort_copy['percentile']=pd.cut(Normal_sort_copy[rad],bins, labels=[1,3,0])
    Normal_sort_drop = pd.concat([Normal_sort_copy[Normal_sort_copy['percentile']==1],Normal_sort_copy[Normal_sort_copy['percentile']==0]],axis = 0)
    
    print('ThIS IS {} SIDE VIEW'.format(key),'\n\n')
    X = Normal_sort_drop.loc[:,0:202]
    y = Normal_sort_drop['percentile'].to_numpy().reshape(-1,1)

    # prepare data (log transform)
    from scipy.stats import skew
    skewness = X.apply(lambda x: skew(x))
    skewed_feats = skewness[skewness > 0.75]
    skewed_feats = skewed_feats.index
    minimum = X[skewed_feats].min(axis = 1).min()
    skewed_feats = np.log1p(X[skewed_feats]+ abs(minimum))
    X = X.to_numpy()

    # Logistic Regression 01 (log transform, Scaler, RFECV, LR)
    pipe01 = Pipeline([('scaler',MinMaxScaler()),
                     ('features',RFECV(estimator = LogisticRegression(solver='liblinear'),cv =3, scoring ='roc_auc')),
                     ('lr',LogisticRegression(solver='liblinear',random_state=42))])
    param01 = {'lr__penalty':['l1','l2'],
            'lr__C':[1000,100,10,1.0,0.1,0.01,0.001]} 

    print('*'*30)
    print('Below is the results from the LR01:')
    self_LR_pipe(X,y,pipe01,param01) 
    print('*'*30)


    # Logistic Regression 03 (log transform, scaler, SelectFromModel, LR)
    selector = SelectFromModel(estimator=LogisticRegression(solver = 'liblinear'))
    pipe_lr03 = Pipeline([ 
            ('scaler',MinMaxScaler()),
            ('features',selector),
            ('lr',LogisticRegression(solver = 'liblinear',random_state=42))]) 
    param_lr03 = {'features__max_features':[20,30,40],
                    'lr__penalty':['l1', 'l2'],
                    'lr__C':[1000,100,10,1.0,0.1,0.01,0.001]} 
    print('*'*30)
    print('Below is the results from the LR03:') 
    self_LR_pipe(X,y,pipe=pipe_lr03,param=param_lr03)
    

# For Chinese Rads Random Forest 01
rad = rads[0]
bins = Bins[0]
for key,value in Side_View.items():
    Normal_sort = value.sort_values(by = rad)
    Normal_sort_copy = Normal_sort.copy()
    Normal_sort_copy['percentile']=pd.cut(Normal_sort_copy[rad],bins, labels=[1,3,0])
    Normal_sort_drop = pd.concat([Normal_sort_copy[Normal_sort_copy['percentile']==1],Normal_sort_copy[Normal_sort_copy['percentile']==0]],axis = 0)
    
    print('ThIS IS {} SIDE VIEW'.format(key),'\n\n')
    X = Normal_sort_drop.loc[:,0:202].to_numpy()
    y = Normal_sort_drop['percentile'].to_numpy().reshape(-1,1)

    # Random forest 01 (scaler, pca + mututal info classifier, RandomForest)
    pca = PCA(n_components = 0.9) 
    selection = SelectKBest(mutual_info_classif,k=10)
    combined_features = FeatureUnion([("pca", pca), ("univ_select", selection)])
    pipe = Pipeline([('scaler',MinMaxScaler()),
                    ('features',combined_features),
                     ('rf',RandomForestClassifier(random_state=42))])

    param = {'features__univ_select__k':[10,20,30,40],
             'rf__max_features':['sqrt','log2'],
             'rf__n_estimators':[50,100,1000,2000]} 
    print('*'*50)
    print('Below is the results from the RF01:')

    cv_outer = LeaveOneOut()
    y_true,y_pred = list(),list()
    for train_ix, test_ix in cv_outer.split(X):
        X_train, X_test = X[train_ix, :], X[test_ix, :]
        y_train, y_test = y[train_ix], y[test_ix]

        ## inner loop for feature selection and hyperparameter tuning 
        cv_inner = StratifiedKFold(n_splits=3, shuffle=True, random_state=24)
        result = GridSearchCV(pipe, param, cv = cv_inner, scoring = 'roc_auc',n_jobs = -1,refit=True).fit(X_train, y_train.ravel())
        best_param = result.best_estimator_
        # evaluate model on the hold out evaluation dataset
        yhat_proba = list(best_param.predict_proba(X_test)[:,1])# reture the probability of predicting '1'
        y_pred.append(yhat_proba)
        y_true.append(list(y_test[0]))
        # report progress
        print('>est=%.3f, cfg=%s' % (result.best_score_, result.best_params_))
        # Calculate roc_auc on the hold out dataset
    print('auc: %.3f' % roc_auc_score(y_true, y_pred))
    print('*'*50)

In [6]:
# # Random Forest 02 (scaler, SelectFromModel, RandomForest)
# rad = rads[0]
# df_cn = dict()
# AUC = dict()
# ACU = dict()
# SEN = dict()
# SPE = dict()
# #bins = Bins[0]
# for key,value in Side_View.items():
#     Normal_sort = value.sort_values(by = rad)
#     Normal_sort_copy = Normal_sort.copy()
#     Normal_sort_copy['percentile']=pd.qcut(Normal_sort_copy[rad],3, labels=[1,3,0])
#     Normal_sort_drop = pd.concat([Normal_sort_copy[Normal_sort_copy['percentile']==1],Normal_sort_copy[Normal_sort_copy['percentile']==0]],axis = 0)
    
#     print('ThIS IS {} SIDE VIEW'.format(key),'\n\n')
#     X = Normal_sort_drop.loc[:,"feat.RA.Normalized.1":"feat.RA.Normalized.203"].to_numpy()
#     y = Normal_sort_drop['percentile'].to_numpy().reshape(-1,1)
#     over = SMOTE(sampling_strategy="minority",random_state=2)
#     X,y = over.fit_resample(X,y)


#     selector = SelectFromModel(estimator=RandomForestClassifier(),max_features = 5)
#     pipe = Pipeline([ #('scaler',MinMaxScaler()),
#                      ('features',selector),
#                      ('rf',RandomForestClassifier(random_state=42))]) 
#     param = {'features__max_features':[5,10,20],
#             "rf__max_depth":[1,5,10],
#              "rf__max_samples":[0.1,0.5,1.0],
#                  'rf__n_estimators':[50,100,1000] } 
#     print('*'*50)
#     Dict,AUC_score,Accuracy,sensitivity1,specificity1 = self_rf_pipe(X,y,pipe,param)  
#     print('*'*50)
#     df_cn[key]=Dict
#     AUC[key]=AUC_score
#     ACU[key]=Accuracy
#     SEN[key]=sensitivity1
#     SPE[key]=specificity1

ThIS IS left_CC SIDE VIEW 


**************************************************
>est=0.867, cfg={'features__max_features': 10, 'rf__max_depth': 5, 'rf__max_samples': 1.0, 'rf__n_estimators': 1000}
>est=0.827, cfg={'features__max_features': 10, 'rf__max_depth': 5, 'rf__max_samples': 0.5, 'rf__n_estimators': 1000}
>est=0.867, cfg={'features__max_features': 20, 'rf__max_depth': 5, 'rf__max_samples': 1.0, 'rf__n_estimators': 1000}
>est=0.860, cfg={'features__max_features': 20, 'rf__max_depth': 5, 'rf__max_samples': 1.0, 'rf__n_estimators': 1000}
>est=0.860, cfg={'features__max_features': 20, 'rf__max_depth': 5, 'rf__max_samples': 1.0, 'rf__n_estimators': 50}
>est=0.893, cfg={'features__max_features': 20, 'rf__max_depth': 1, 'rf__max_samples': 0.5, 'rf__n_estimators': 1000}
>est=0.890, cfg={'features__max_features': 20, 'rf__max_depth': 5, 'rf__max_samples': 0.5, 'rf__n_estimators': 50}
>est=0.920, cfg={'features__max_features': 5, 'rf__max_depth': 5, 'rf__max_samples': 1.0, 'rf__n_estimato

>est=0.843, cfg={'features__max_features': 20, 'rf__max_depth': 5, 'rf__max_samples': 0.5, 'rf__n_estimators': 50}
2 : 10
3 : 6
5 : 6
6 : 4
7 : 13
8 : 6
9 : 2
10 : 5
11 : 3
15 : 7
36 : 9
37 : 11
38 : 15
39 : 21
40 : 10
41 : 5
42 : 8
43 : 9
70 : 1
75 : 2
82 : 1
83 : 1
110 : 24
112 : 16
114 : 7
121 : 8
122 : 5
123 : 6
124 : 8
125 : 8
126 : 7
127 : 18
128 : 17
129 : 16
130 : 5
131 : 9
132 : 13
137 : 3
139 : 7
140 : 5
175 : 11
176 : 4
195 : 1
199 : 10
201 : 2
[38, 39, 110, 112, 127, 128, 129]
auc: 0.820
Accuracy:  0.7666666666666667
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0.94, 0.72, 0.96, 1.0, 0.92, 0.55, 0.92, 0.12, 0.42, 0.0, 0.5051587301587301, 0.9, 0.94, 0.93, 0.82, 0.71, 0.22, 0.28, 0.0, 0.816, 0.17, 0.64, 0.44, 0.52, 0.48, 0.013, 0.12, 0.2, 0.0, 0.46]
Sensitivity :  0.7333333333333333
Specificity :  0.8
**************************************************
ThIS IS left_MLO SIDE VIEW 


*************************************************

>est=0.973, cfg={'features__max_features': 5, 'rf__max_depth': 1, 'rf__max_samples': 1.0, 'rf__n_estimators': 1000}
>est=0.993, cfg={'features__max_features': 5, 'rf__max_depth': 5, 'rf__max_samples': 0.1, 'rf__n_estimators': 100}
>est=0.973, cfg={'features__max_features': 5, 'rf__max_depth': 1, 'rf__max_samples': 0.1, 'rf__n_estimators': 1000}
>est=0.987, cfg={'features__max_features': 10, 'rf__max_depth': 5, 'rf__max_samples': 0.5, 'rf__n_estimators': 1000}
>est=0.993, cfg={'features__max_features': 5, 'rf__max_depth': 5, 'rf__max_samples': 0.1, 'rf__n_estimators': 100}
>est=0.987, cfg={'features__max_features': 10, 'rf__max_depth': 10, 'rf__max_samples': 0.1, 'rf__n_estimators': 1000}
>est=0.987, cfg={'features__max_features': 5, 'rf__max_depth': 1, 'rf__max_samples': 0.1, 'rf__n_estimators': 50}
2 : 18
3 : 12
5 : 4
6 : 9
7 : 3
8 : 2
9 : 2
11 : 2
36 : 20
37 : 22
38 : 27
39 : 17
40 : 1
41 : 6
42 : 6
43 : 7
55 : 1
69 : 1
73 : 1
74 : 1
75 : 2
82 : 1
83 : 2
110 : 2
112 : 1
123 : 2
130 :

In [7]:
# PIPELINE EVALUATE FOR CHINESE 
rad = rads[0]
PROB = []
PRED = []
TRUE = []
for key,value in Side_View.items():
    Normal_sort = value.sort_values(by = rad)
    Normal_sort_copy = Normal_sort.copy()
    Normal_sort_copy['percentile']=pd.qcut(Normal_sort_copy[rad],3, labels=[0,3,1])
    Normal_sort_drop = pd.concat([Normal_sort_copy[Normal_sort_copy['percentile']==0],Normal_sort_copy[Normal_sort_copy['percentile']==1]],axis = 0)
    print('ThIS IS {} SIDE VIEW'.format(key),'\n\n')
    X = Normal_sort_drop.loc[:,"feat.RA.Normalized.1":"feat.RA.Normalized.203"].to_numpy()
    y = Normal_sort_drop['percentile'].to_numpy().reshape(-1,1)
    selector = SelectFromModel(estimator=RandomForestClassifier(),max_features = 5)
    pipe = Pipeline([('scaler',MinMaxScaler()),
                     ('features',selector),
                     ('rf',RandomForestClassifier(random_state=42))]) 
    param = {"features__max_features":[5,10,20],
            "rf__max_depth":[1,5,10],
             "rf__max_samples":[0.1,0.5,1.0],
            "rf__n_estimators":[50,100,1000] } 
    probability, pre_label,true_value = pipeline_evaluate(X,y,pipe, param)
    AUC_score = roc_auc_score(true_value, probability)
    Accuracy = accuracy_score(true_value,pre_label)
    
    cm1 = confusion_matrix(true_value,pre_label)
    sensitivity1 = cm1[0,0]/(cm1[0,0]+cm1[0,1])
    specificity1 = cm1[1,1]/(cm1[1,0]+cm1[1,1])
    print('Specificity : ', specificity1)
    print("The AUC for this view is :", AUC_score)
    print("The ACC for this view is :", Accuracy)
    print("The sen for this view is :", sensitivity1)
    print("The spe for this view is :", specificity1)
    
    PROB.append(probability)
    PRED.append(pre_label)

ThIS IS left_CC SIDE VIEW 


>est=0.871, cfg={'features__max_features': 5, 'rf__max_depth': 5, 'rf__max_samples': 0.1, 'rf__n_estimators': 100}
>est=0.872, cfg={'features__max_features': 20, 'rf__max_depth': 10, 'rf__max_samples': 0.1, 'rf__n_estimators': 1000}
>est=0.861, cfg={'features__max_features': 20, 'rf__max_depth': 5, 'rf__max_samples': 0.1, 'rf__n_estimators': 100}
>est=0.857, cfg={'features__max_features': 20, 'rf__max_depth': 1, 'rf__max_samples': 0.5, 'rf__n_estimators': 1000}
>est=0.874, cfg={'features__max_features': 20, 'rf__max_depth': 5, 'rf__max_samples': 0.5, 'rf__n_estimators': 1000}
>est=0.890, cfg={'features__max_features': 20, 'rf__max_depth': 10, 'rf__max_samples': 0.1, 'rf__n_estimators': 50}
>est=0.878, cfg={'features__max_features': 10, 'rf__max_depth': 10, 'rf__max_samples': 0.1, 'rf__n_estimators': 1000}
>est=0.907, cfg={'features__max_features': 20, 'rf__max_depth': 10, 'rf__max_samples': 0.5, 'rf__n_estimators': 50}
>est=0.906, cfg={'features__max_featur

>est=0.906, cfg={'features__max_features': 20, 'rf__max_depth': 10, 'rf__max_samples': 1.0, 'rf__n_estimators': 1000}
>est=0.913, cfg={'features__max_features': 20, 'rf__max_depth': 10, 'rf__max_samples': 1.0, 'rf__n_estimators': 100}
>est=0.898, cfg={'features__max_features': 20, 'rf__max_depth': 5, 'rf__max_samples': 1.0, 'rf__n_estimators': 1000}
>est=0.896, cfg={'features__max_features': 20, 'rf__max_depth': 5, 'rf__max_samples': 1.0, 'rf__n_estimators': 1000}
>est=0.915, cfg={'features__max_features': 5, 'rf__max_depth': 5, 'rf__max_samples': 1.0, 'rf__n_estimators': 1000}
>est=0.897, cfg={'features__max_features': 20, 'rf__max_depth': 10, 'rf__max_samples': 1.0, 'rf__n_estimators': 50}
>est=0.917, cfg={'features__max_features': 5, 'rf__max_depth': 1, 'rf__max_samples': 1.0, 'rf__n_estimators': 100}
>est=0.904, cfg={'features__max_features': 5, 'rf__max_depth': 5, 'rf__max_samples': 1.0, 'rf__n_estimators': 1000}
>est=0.898, cfg={'features__max_features': 10, 'rf__max_depth': 10, 

In [8]:
pipeline_prob_cn = np.array(PROB).reshape(4,-1)
agg_prob_cn = np.max(pipeline_prob_cn, axis=0)

label = []
for i in range(len(agg_prob_cn)):
    if agg_prob_cn[i]>0.5:
        label.append(1)
    else:
        label.append(0)
         
# calculate sensitivity, specificity, accuracy
Accuracy = accuracy_score(true_value,label)
cm1 = confusion_matrix(true_value,label)
sensitivity1 = cm1[0,0]/(cm1[0,0]+cm1[0,1])
specificity1 = cm1[1,1]/(cm1[1,0]+cm1[1,1])

print("Accuray is :",Accuracy )
print("Sensitivity is :", sensitivity1)
print("Specificity is :", specificity1)

# calculate pipeline AUC
AUC_score = roc_auc_score(true_value, agg_prob_cn)
print("AUC for normal pipeline for Chinese readers is {:.3f}".format(AUC_score))

# get confidence interval
n_bootstraps = 2000
rng_seed = 42  # control reproducibility
bootstrapped_scores = []

rng = np.random.RandomState(rng_seed)
for i in range(n_bootstraps):
    # bootstrap by sampling with replacement on the prediction indices
    indices = rng.randint(0, len(agg_prob_cn), len(agg_prob_cn))
    if len(np.unique(np.array(true_value)[indices])) < 2:
        # We need at least one positive and one negative sample for ROC AUC
        # to be defined: reject the sample
        continue

    score = roc_auc_score(np.array(true_value)[indices], agg_prob_cn[indices])
    bootstrapped_scores.append(score)
#     print("Bootstrap #{} ROC area: {:0.3f}".format(i + 1, score))

sorted_scores = np.array(bootstrapped_scores)
sorted_scores.sort()
confidence_lower = sorted_scores[int(0.025 * len(sorted_scores))]
confidence_upper = sorted_scores[int(0.975 * len(sorted_scores))]
print("Confidence interval for the score: [{:0.3f} - {:0.3}]".format(
    confidence_lower, confidence_upper))

Accuray is : 0.7037037037037037
Sensitivity is : 0.5333333333333333
Specificity is : 0.9166666666666666
AUC for normal pipeline for Chinese readers is 0.856
Confidence interval for the score: [0.676 - 0.987]


## Australian Normal

# logistic Regression 02 (log, pca + mutual_info, LR) 
rad = rads[1]
bins = Bins[1]
print('Below is for Australian Radiologists: \n')
for key,value in Side_View.items():
    Normal_sort = value.sort_values(by = rad)
    Normal_sort_copy = Normal_sort.copy()
    Normal_sort_copy['percentile']=pd.cut(Normal_sort_copy[rad],bins, labels=[1,3,0])
    Normal_sort_drop = pd.concat([Normal_sort_copy[Normal_sort_copy['percentile']==1],Normal_sort_copy[Normal_sort_copy['percentile']==0]],axis = 0)
    
    print('ThIS IS {} SIDE VIEW'.format(key),'\n\n')
    X = Normal_sort_drop.loc[:,0:202]
    y = Normal_sort_drop['percentile'].to_numpy().reshape(-1,1)

    # prepare data (log transform)
    from scipy.stats import skew
    skewness = X.apply(lambda x: skew(x))
    skewed_feats = skewness[skewness > 0.75]
    skewed_feats = skewed_feats.index
    minimum = X[skewed_feats].min(axis = 1).min()
    skewed_feats = np.log1p(X[skewed_feats]+ abs(minimum))
    X = X.to_numpy()
    pca = PCA(n_components = 0.9)
    selection = SelectKBest(mutual_info_classif,k=10)
    combined_features = FeatureUnion([("pca", pca), ("univ_select", selection)])

    pipe_lr2 = Pipeline([
        ('scaler',MinMaxScaler()),
        ('features',combined_features),
        ('lr',LogisticRegression(solver='liblinear',random_state=42))])
    param_lr2 = {'features__univ_select__k':[5,10,20,30],
                'lr__penalty':['l1', 'l2'],
                'lr__C':[1000,100,10,1.0,0.1,0.01,0.001]} 

    print('Below is the results from the LR02:')
    cv_outer = LeaveOneOut()
    y_true,y_pred = list(),list()
    for train_ix, test_ix in cv_outer.split(X):
        X_train_, X_test = X[train_ix, :], X[test_ix, :]
        y_train_, y_test = y[train_ix], y[test_ix]

        # inner loop for feature selection and hyperparameter tuning 
        cv_inner = StratifiedKFold(n_splits=3, shuffle=True, random_state=24)
        result = GridSearchCV(pipe_lr2, param_lr2, cv = cv_inner, n_jobs= -1,scoring = 'roc_auc',refit=True).fit(X_train_, y_train_.ravel())
        best_param = result.best_estimator_
        # evaluate model on the hold out validation set
        yhat_proba = list(best_param.predict_proba(X_test)[:,1])
        y_pred.append(yhat_proba)
        y_true.append(list(y_test[0]))
        # report progress
        print('>est=%.3f, cfg=%s' % (result.best_score_, result.best_params_))
       # Calculate roc_auc on the hold out dataset
    print('auc: %.3f' % roc_auc_score(y_true, y_pred))       

    print('*'*30)

# For Australian Rads Logistic regression 1 3
rad = rads[1]
bins = Bins[1]
print('Below is for Australian Radiologists: \n')
for key,value in Side_View.items():
    Normal_sort = value.sort_values(by = rad)
    Normal_sort_copy = Normal_sort.copy()
    Normal_sort_copy['percentile']=pd.cut(Normal_sort_copy[rad],bins, labels=[1,3,0])
    Normal_sort_drop = pd.concat([Normal_sort_copy[Normal_sort_copy['percentile']==1],Normal_sort_copy[Normal_sort_copy['percentile']==0]],axis = 0)
    
    print('ThIS IS {} SIDE VIEW'.format(key),'\n\n')
    X = Normal_sort_drop.loc[:,0:202]
    y = Normal_sort_drop['percentile'].to_numpy().reshape(-1,1)


    # prepare data (log transform)
    from scipy.stats import skew
    skewness = X.apply(lambda x: skew(x))
    skewed_feats = skewness[skewness > 0.75]
    skewed_feats = skewed_feats.index
    minimum = X[skewed_feats].min(axis = 1).min()
    skewed_feats = np.log1p(X[skewed_feats]+ abs(minimum))
    X = X.to_numpy()

    # Logistic Regression 01 (log transform, Scaler, RFECV, LR)
    pipe01 = Pipeline([('scaler',MinMaxScaler()),
                     ('features',RFECV(estimator = LogisticRegression(solver='liblinear'),cv =3, scoring ='roc_auc')),
                     ('lr',LogisticRegression(solver='liblinear',random_state=42))])
    param01 = {'lr__penalty':['l1','l2'],
            'lr__C':[1000,100,10,1.0,0.1,0.01,0.001]} 
    print('*'*30)
    print('Below is the results from the LR01:')
    self_LR_pipe(X,y,pipe01,param01)
    print('*'*30)


    # Logistic Regression 03 (log transform, scaler, SelectFromModel, LR)
    selector = SelectFromModel(estimator=LogisticRegression(solver = 'liblinear'))
    pipe_lr03 = Pipeline([ 
            ('scaler',MinMaxScaler()),
        ('features',selector),
        ('lr',LogisticRegression(solver = 'liblinear',random_state=42))]) 
    param_lr03 = {'features__max_features':[20,30,40],
                    'lr__penalty':['l1', 'l2'],
                    'lr__C':[1000,100,10,1.0,0.1,0.01,0.001]} 
    print('*'*30)
    print('Below is the results from the LR03:')               
    self_LR_pipe(X,y,pipe=pipe_lr03,param=param_lr03)
    print('*'*30)

  

# For Australian Rads Random Forest 01
rad = rads[1]
bins = Bins[1]
for key,value in Side_View.items():
    Normal_sort = value.sort_values(by = rad)
    Normal_sort_copy = Normal_sort.copy()
    Normal_sort_copy['percentile']=pd.cut(Normal_sort_copy[rad],bins, labels=[1,3,0])
    Normal_sort_drop = pd.concat([Normal_sort_copy[Normal_sort_copy['percentile']==1],Normal_sort_copy[Normal_sort_copy['percentile']==0]],axis = 0)
    
    print('ThIS IS {} SIDE VIEW'.format(key),'\n\n')
    X = Normal_sort_drop.loc[:,0:202].to_numpy()
    y = Normal_sort_drop['percentile'].to_numpy().reshape(-1,1)



    # Random forest 01 (scaler, pca + mututal info classifier, RandomForest)
    pca = PCA(n_components = 0.9) 
    selection = SelectKBest(mutual_info_classif,k=10)
    combined_features = FeatureUnion([("pca", pca), ("univ_select", selection)])
    pipe = Pipeline([('scaler',MinMaxScaler()),
                    ('features',combined_features),
                     ('rf',RandomForestClassifier(random_state=42))])

    param = {'features__univ_select__k':[10,20,30,40],
             'rf__max_features':['sqrt','log2'],
             'rf__n_estimators':[50,100,1000,2000]} 


    print('*'*50)
    print('Below is the results from the RF01:')
    cv_outer = LeaveOneOut()
    y_true,y_pred = list(),list()
    for train_ix, test_ix in cv_outer.split(X):
        X_train, X_test = X[train_ix, :], X[test_ix, :]
        y_train, y_test = y[train_ix], y[test_ix]

        ## inner loop for feature selection and hyperparameter tuning 
        cv_inner = StratifiedKFold(n_splits=3, shuffle=True, random_state=24)
        result = GridSearchCV(pipe, param, cv = cv_inner, scoring = 'roc_auc',n_jobs = -1,refit=True).fit(X_train, y_train.ravel())
        best_param = result.best_estimator_
        # evaluate model on the hold out evaluation dataset
        yhat_proba = list(best_param.predict_proba(X_test)[:,1])# reture the probability of predicting '1'
        y_pred.append(yhat_proba)
        y_true.append(list(y_test[0]))
        # report progress
        print('>est=%.3f, cfg=%s' % (result.best_score_, result.best_params_))
        # Calculate roc_auc on the hold out dataset
    print('auc: %.3f' % roc_auc_score(y_true, y_pred))
    print('*'*50)

In [7]:
# # Random Forest 02 (scaler, SelectFromModel, RandomForest)
# rad = rads[1]
# df_au = dict()
# AUC = dict()
# ACU = dict()
# SEN = dict()
# SPE = dict()
# #bins = Bins[0]
# for key,value in Side_View.items():
#     Normal_sort = value.sort_values(by = rad)
#     Normal_sort_copy = Normal_sort.copy()
#     Normal_sort_copy['percentile']=pd.qcut(Normal_sort_copy[rad],3, labels=[1,3,0])
#     Normal_sort_drop = pd.concat([Normal_sort_copy[Normal_sort_copy['percentile']==1],Normal_sort_copy[Normal_sort_copy['percentile']==0]],axis = 0)
    
#     print('ThIS IS {} SIDE VIEW'.format(key),'\n\n')
#     X = Normal_sort_drop.loc[:,"feat.RA.Normalized.1":"feat.RA.Normalized.203"].to_numpy()
#     y = Normal_sort_drop['percentile'].to_numpy().reshape(-1,1)
#     over = SMOTE(sampling_strategy="minority",random_state=2)
#     X,y = over.fit_resample(X,y)
  

#     selector = SelectFromModel(estimator=RandomForestClassifier(),max_features = 5)
#     pipe = Pipeline([ #('scaler',MinMaxScaler()),
#                      ('features',selector),
#                      ('rf',RandomForestClassifier(random_state=42))]) 
#     param = {'features__max_features':[5,10,20],
#             "rf__max_depth":[1,5,10],
#              "rf__max_samples":[0.1,0.5,1.0],
#                  'rf__n_estimators':[50,100,1000] } 
#     print('*'*50)
#     Dict,AUC_score,Accuracy,sensitivity1,specificity1 = self_rf_pipe(X,y,pipe,param)  
#     print('*'*50)
#     df_au[key]=Dict
#     AUC[key]=AUC_score
#     ACU[key]=Accuracy
#     SEN[key]=sensitivity1
#     SPE[key]=specificity1

ThIS IS left_CC SIDE VIEW 


**************************************************
>est=0.812, cfg={'features__max_features': 5, 'rf__max_depth': 5, 'rf__max_samples': 1.0, 'rf__n_estimators': 100}
>est=0.796, cfg={'features__max_features': 5, 'rf__max_depth': 10, 'rf__max_samples': 0.5, 'rf__n_estimators': 1000}
>est=0.767, cfg={'features__max_features': 10, 'rf__max_depth': 5, 'rf__max_samples': 1.0, 'rf__n_estimators': 100}
>est=0.874, cfg={'features__max_features': 5, 'rf__max_depth': 5, 'rf__max_samples': 1.0, 'rf__n_estimators': 50}
>est=0.772, cfg={'features__max_features': 5, 'rf__max_depth': 10, 'rf__max_samples': 0.5, 'rf__n_estimators': 1000}
>est=0.807, cfg={'features__max_features': 5, 'rf__max_depth': 5, 'rf__max_samples': 1.0, 'rf__n_estimators': 1000}
>est=0.770, cfg={'features__max_features': 5, 'rf__max_depth': 10, 'rf__max_samples': 0.1, 'rf__n_estimators': 50}
>est=0.749, cfg={'features__max_features': 10, 'rf__max_depth': 10, 'rf__max_samples': 1.0, 'rf__n_estimators'

>est=0.713, cfg={'features__max_features': 5, 'rf__max_depth': 5, 'rf__max_samples': 1.0, 'rf__n_estimators': 1000}
>est=0.782, cfg={'features__max_features': 5, 'rf__max_depth': 10, 'rf__max_samples': 1.0, 'rf__n_estimators': 1000}
>est=0.687, cfg={'features__max_features': 20, 'rf__max_depth': 5, 'rf__max_samples': 0.5, 'rf__n_estimators': 1000}
>est=0.713, cfg={'features__max_features': 20, 'rf__max_depth': 5, 'rf__max_samples': 1.0, 'rf__n_estimators': 1000}
>est=0.785, cfg={'features__max_features': 5, 'rf__max_depth': 10, 'rf__max_samples': 0.1, 'rf__n_estimators': 50}
>est=0.755, cfg={'features__max_features': 10, 'rf__max_depth': 5, 'rf__max_samples': 0.5, 'rf__n_estimators': 100}
>est=0.796, cfg={'features__max_features': 5, 'rf__max_depth': 10, 'rf__max_samples': 1.0, 'rf__n_estimators': 1000}
>est=0.687, cfg={'features__max_features': 5, 'rf__max_depth': 5, 'rf__max_samples': 0.5, 'rf__n_estimators': 1000}
>est=0.663, cfg={'features__max_features': 10, 'rf__max_depth': 10, '

**************************************************
>est=0.664, cfg={'features__max_features': 10, 'rf__max_depth': 5, 'rf__max_samples': 1.0, 'rf__n_estimators': 1000}
>est=0.696, cfg={'features__max_features': 5, 'rf__max_depth': 1, 'rf__max_samples': 1.0, 'rf__n_estimators': 50}
>est=0.675, cfg={'features__max_features': 5, 'rf__max_depth': 10, 'rf__max_samples': 1.0, 'rf__n_estimators': 1000}
>est=0.688, cfg={'features__max_features': 5, 'rf__max_depth': 5, 'rf__max_samples': 0.5, 'rf__n_estimators': 50}
>est=0.694, cfg={'features__max_features': 10, 'rf__max_depth': 5, 'rf__max_samples': 0.1, 'rf__n_estimators': 50}
>est=0.642, cfg={'features__max_features': 5, 'rf__max_depth': 5, 'rf__max_samples': 0.5, 'rf__n_estimators': 50}
>est=0.590, cfg={'features__max_features': 20, 'rf__max_depth': 1, 'rf__max_samples': 1.0, 'rf__n_estimators': 50}
>est=0.664, cfg={'features__max_features': 20, 'rf__max_depth': 5, 'rf__max_samples': 0.1, 'rf__n_estimators': 50}
>est=0.599, cfg={'features__

In [9]:
# PIPELINE EVALUATE FOR AUSTRALIAN 
rad = rads[1]
PROB = []
PRED = []
TRUE = []
for key,value in Side_View.items():
    Normal_sort = value.sort_values(by = rad)
    Normal_sort_copy = Normal_sort.copy()
    Normal_sort_copy['percentile']=pd.qcut(Normal_sort_copy[rad],3, labels=[0,3,1])
    Normal_sort_drop = pd.concat([Normal_sort_copy[Normal_sort_copy['percentile']==0],Normal_sort_copy[Normal_sort_copy['percentile']==1]],axis = 0)
    print('ThIS IS {} SIDE VIEW'.format(key),'\n\n')
    X = Normal_sort_drop.loc[:,"feat.RA.Normalized.1":"feat.RA.Normalized.203"].to_numpy()
    y = Normal_sort_drop['percentile'].to_numpy().reshape(-1,1)
    selector = SelectFromModel(estimator=RandomForestClassifier(),max_features = 5)
    pipe = Pipeline([('scaler',MinMaxScaler()),
                     ('features',selector),
                     ('rf',RandomForestClassifier(random_state=42))]) 
    param = {"features__max_features":[5,10,20],
            "rf__max_depth":[1,5,10],
             "rf__max_samples":[0.1,0.5,1.0],
            "rf__n_estimators":[50,100,1000] } 
    probability, pre_label,true_value = pipeline_evaluate(X,y,pipe, param)
    AUC_score = roc_auc_score(true_value, probability)
    Accuracy = accuracy_score(true_value,pre_label)
    cm1 = confusion_matrix(true_value,pre_label)
    sensitivity1 = cm1[0,0]/(cm1[0,0]+cm1[0,1])
    specificity1 = cm1[1,1]/(cm1[1,0]+cm1[1,1])
    print('Specificity : ', specificity1)
    print("The AUC for this view is :", AUC_score)
    print("The ACC for this view is :", Accuracy)
    print("The sen for this view is :", sensitivity1)
    print("The spe for this view is :", specificity1)
    
    PROB.extend(probability)
    PRED.append(pre_label)

ThIS IS left_CC SIDE VIEW 


>est=0.534, cfg={'features__max_features': 5, 'rf__max_depth': 10, 'rf__max_samples': 0.1, 'rf__n_estimators': 100}
>est=0.502, cfg={'features__max_features': 5, 'rf__max_depth': 1, 'rf__max_samples': 1.0, 'rf__n_estimators': 100}
>est=0.456, cfg={'features__max_features': 10, 'rf__max_depth': 1, 'rf__max_samples': 0.5, 'rf__n_estimators': 100}
>est=0.530, cfg={'features__max_features': 20, 'rf__max_depth': 1, 'rf__max_samples': 1.0, 'rf__n_estimators': 50}
>est=0.497, cfg={'features__max_features': 20, 'rf__max_depth': 1, 'rf__max_samples': 0.1, 'rf__n_estimators': 50}
>est=0.495, cfg={'features__max_features': 20, 'rf__max_depth': 5, 'rf__max_samples': 0.1, 'rf__n_estimators': 50}
>est=0.545, cfg={'features__max_features': 10, 'rf__max_depth': 1, 'rf__max_samples': 0.5, 'rf__n_estimators': 1000}
>est=0.535, cfg={'features__max_features': 5, 'rf__max_depth': 10, 'rf__max_samples': 0.1, 'rf__n_estimators': 1000}
>est=0.504, cfg={'features__max_features': 20

>est=0.630, cfg={'features__max_features': 10, 'rf__max_depth': 10, 'rf__max_samples': 0.1, 'rf__n_estimators': 100}
>est=0.539, cfg={'features__max_features': 10, 'rf__max_depth': 1, 'rf__max_samples': 0.1, 'rf__n_estimators': 1000}
>est=0.579, cfg={'features__max_features': 5, 'rf__max_depth': 1, 'rf__max_samples': 0.1, 'rf__n_estimators': 100}
>est=0.556, cfg={'features__max_features': 5, 'rf__max_depth': 10, 'rf__max_samples': 0.5, 'rf__n_estimators': 100}
>est=0.579, cfg={'features__max_features': 5, 'rf__max_depth': 1, 'rf__max_samples': 0.1, 'rf__n_estimators': 100}
>est=0.529, cfg={'features__max_features': 5, 'rf__max_depth': 5, 'rf__max_samples': 0.1, 'rf__n_estimators': 50}
>est=0.511, cfg={'features__max_features': 20, 'rf__max_depth': 1, 'rf__max_samples': 0.1, 'rf__n_estimators': 100}
>est=0.520, cfg={'features__max_features': 5, 'rf__max_depth': 5, 'rf__max_samples': 0.1, 'rf__n_estimators': 1000}
>est=0.525, cfg={'features__max_features': 5, 'rf__max_depth': 5, 'rf__max

In [10]:
PROB = np.array(PROB).reshape(4,-1)
PROB_max = np.max(PROB,axis=0)

label = []
for i in range(len(PROB_max)):
    if PROB_max[i]>0.5:
        label.append(1)
    else:
        label.append(0)
         
# calculate sensitivity, specificity, accuracy
Accuracy = accuracy_score(true_value,label)
cm1 = confusion_matrix(true_value,label)
sensitivity1 = cm1[0,0]/(cm1[0,0]+cm1[0,1])
specificity1 = cm1[1,1]/(cm1[1,0]+cm1[1,1])

print("Accuray is :",Accuracy )
print("Sensitivity is :", sensitivity1)
print("Specificity is :", specificity1)

# calculate pipeline AUC
AUC_score = roc_auc_score(true_value, PROB_max)
print("AUC for normal pipeline for Australian readers is {:.3f}".format(AUC_score))

# get confidence interval
n_bootstraps = 2000
rng_seed = 42  # control reproducibility
bootstrapped_scores = []

rng = np.random.RandomState(rng_seed)
for i in range(n_bootstraps):
    # bootstrap by sampling with replacement on the prediction indices
    indices = rng.randint(0, len(PROB_max), len(PROB_max))
    if len(np.unique(np.array(true_value)[indices])) < 2:
        # We need at least one positive and one negative sample for ROC AUC
        # to be defined: reject the sample
        continue

    score = roc_auc_score(np.array(true_value)[indices], PROB_max[indices])
    bootstrapped_scores.append(score)
#     print("Bootstrap #{} ROC area: {:0.3f}".format(i + 1, score))

sorted_scores = np.array(bootstrapped_scores)
sorted_scores.sort()
confidence_lower = sorted_scores[int(0.025 * len(sorted_scores))]
confidence_upper = sorted_scores[int(0.975 * len(sorted_scores))]
print("Confidence interval for the score: [{:0.3f} - {:0.3}]".format(
    confidence_lower, confidence_upper))

Accuray is : 0.3333333333333333
Sensitivity is : 0.3157894736842105
Specificity is : 0.36363636363636365
AUC for normal pipeline for Australian readers is 0.316
Confidence interval for the score: [0.111 - 0.551]


## Case Based 
### Cancer Lesion Side

In [11]:
# Cancer lesion side  case based
Side_View = {}
Side_View = {'Case_based_CC_CancerLesion':CC_Cancer,
             'Case_based_MLO_CancerLesion':MLO_Cancer}
rads = ['diffScore.CN','diffScore.AU']

# CN and AU Logistic Regression 02
for index, rad in enumerate(rads):
    for key,value in Side_View.items():
        cancer_sort = value.sort_values(by = rad)
        cancer_sort_copy = cancer_sort.copy()
        cancer_sort_copy['percentile'] = pd.cut(cancer_sort_copy[rad], Bins[index], labels=[1,3,0])
        cancer_sort_drop = pd.concat([cancer_sort_copy[cancer_sort_copy['percentile']==1],cancer_sort_copy[cancer_sort_copy['percentile']==0]],axis = 0)

        print('ThIS IS {} SIDE VIEW'.format(key),'\n\n')
        X = cancer_sort_drop.loc[:,0:202]
        y = cancer_sort_drop['percentile'].to_numpy().reshape(-1,1)

        # prepare data (log transform)
        from scipy.stats import skew
        skewness = X.apply(lambda x: skew(x))
        skewed_feats = skewness[skewness > 0.75]
        skewed_feats = skewed_feats.index
        minimum = X[skewed_feats].min(axis = 1).min()
        skewed_feats = np.log1p(X[skewed_feats]+ abs(minimum))
        X = X.to_numpy()  

        pipe_lr2 = Pipeline([
            ('scaler',MinMaxScaler()),
            ('features',combined_features),
            ('lr',LogisticRegression(solver='liblinear',random_state=42))])
        param_lr2 = {'features__univ_select__k':[5,10,20,30],
                    'lr__penalty':['l1', 'l2'],
                    'lr__C':[1000,100,10,1.0,0.1,0.01,0.001]} 
        print('*'*30)
        print('Below is the results from the LR02:')
        cv_outer = LeaveOneOut()
        y_true,y_pred = list(),list()
        for train_ix, test_ix in cv_outer.split(X):
            X_train_, X_test = X[train_ix, :], X[test_ix, :]
            y_train_, y_test = y[train_ix], y[test_ix]

            # inner loop for feature selection and hyperparameter tuning 
            cv_inner = StratifiedKFold(n_splits=3, shuffle=True, random_state=24)
            result = GridSearchCV(pipe_lr2, param_lr2, cv = cv_inner, n_jobs= -1,scoring = 'roc_auc',refit=True).fit(X_train_, y_train_.ravel())
            best_param = result.best_estimator_
            # evaluate model on the hold out validation set
            yhat_proba = list(best_param.predict_proba(X_test)[:,1])
            y_pred.append(yhat_proba)
            y_true.append(list(y_test[0]))
            # report progress
            print('>est=%.3f, cfg=%s' % (result.best_score_, result.best_params_))
           # Calculate roc_auc on the hold out dataset
        print('auc: %.3f' % roc_auc_score(y_true, y_pred))       
        print('*'*30)

# CN and AU Logistic Regression 1 3 
for index, rad in enumerate(rads):
    for key,value in Side_View.items():
        cancer_sort = value.sort_values(by = rad)
        cancer_sort_copy = cancer_sort.copy()
        cancer_sort_copy['percentile'] = pd.cut(cancer_sort_copy[rad], Bins[index], labels=[1,3,0])
        cancer_sort_drop = pd.concat([cancer_sort_copy[cancer_sort_copy['percentile']==1],cancer_sort_copy[cancer_sort_copy['percentile']==0]],axis = 0)

        print('ThIS IS {} SIDE VIEW'.format(key),'\n\n')
        X = cancer_sort_drop.loc[:,0:202]
        y = cancer_sort_drop['percentile'].to_numpy().reshape(-1,1)


        # prepare data (log transform)
        from scipy.stats import skew
        skewness = X.apply(lambda x: skew(x))
        skewed_feats = skewness[skewness > 0.75]
        skewed_feats = skewed_feats.index
        minimum = X[skewed_feats].min(axis = 1).min()
        skewed_feats = np.log1p(X[skewed_feats]+ abs(minimum))
        X = X.to_numpy()
        
        # Logistic Regression 01 (log transform, Scaler, RFECV, LR)
        pipe01 = Pipeline([('scaler',MinMaxScaler()),
                         ('features',RFECV(estimator = LogisticRegression(solver='liblinear'),cv =3, scoring ='roc_auc')),
                         ('lr',LogisticRegression(solver='liblinear',random_state=42))])
        param01 = {'lr__penalty':['l1','l2'],
                'lr__C':[1000,100,10,1.0,0.1,0.01,0.001]} 
        print('*'*30)
        print('Below is the results from the LR01:')
        self_LR_pipe(X,y,pipe01,param01)
        print('*'*30)
        
        # Logistic Regression 03 (log transform, scaler, SelectFromModel, LR)
        selector = SelectFromModel(estimator=LogisticRegression(solver = 'liblinear'))
        pipe_lr03 = Pipeline([ 
                ('scaler',MinMaxScaler()),('features',selector),
            ('lr',LogisticRegression(solver = 'liblinear',random_state=42))]) 
        param_lr03 = {'features__max_features':[20,30,40],
                        'lr__penalty':['l1', 'l2'],
                        'lr__C':[1000,100,10,1.0,0.1,0.01,0.001]} 
        print('*'*30)
        print('Below is the results from the LR03:')               
        self_LR_pipe(X,y,pipe=pipe_lr03,param=param_lr03)
        print('*'*30)

# CN and AU Random Forest01
for index, rad in enumerate(rads):
    for key,value in Side_View.items():
        cancer_sort = value.sort_values(by = rad)
        cancer_sort_copy = cancer_sort.copy()
        cancer_sort_copy['percentile'] = pd.cut(cancer_sort_copy[rad], Bins[index], labels=[1,3,0])
        cancer_sort_drop = pd.concat([cancer_sort_copy[cancer_sort_copy['percentile']==1],cancer_sort_copy[cancer_sort_copy['percentile']==0]],axis = 0)

        print('ThIS IS {} SIDE VIEW'.format(key),'\n\n')
        X = cancer_sort_drop.loc[:,0:202].to_numpy()
        y = cancer_sort_drop['percentile'].to_numpy().reshape(-1,1)


        # Random forest 01 (scaler, pca + mututal info classifier, RandomForest)
        pca = PCA(n_components = 0.9) 
        selection = SelectKBest(mutual_info_classif,k=10)
        combined_features = FeatureUnion([("pca", pca), ("univ_select", selection)])
        pipe = Pipeline([ ('scaler',MinMaxScaler()),
                         ('features',combined_features),
                         ('rf',RandomForestClassifier(random_state=42))]) 
        param = {'features__univ_select__k':[10,20,30,40],
                 'rf__max_features':['sqrt','log2'],
                 'rf__n_estimators':[50,100,1000,2000]} 
        print('*'*50)
        print('Below is the results from the RF01:')
        cv_outer = LeaveOneOut()
        y_true,y_pred = list(),list()
        for train_ix, test_ix in cv_outer.split(X):
            X_train, X_test = X[train_ix, :], X[test_ix, :]
            y_train, y_test = y[train_ix], y[test_ix]

            ## inner loop for feature selection and hyperparameter tuning 
            cv_inner = StratifiedKFold(n_splits=3, shuffle=True, random_state=24)
            result = GridSearchCV(pipe, param, cv = cv_inner, scoring = 'roc_auc',n_jobs = -1,refit=True).fit(X_train, y_train.ravel())
            best_param = result.best_estimator_
            # evaluate model on the hold out evaluation dataset
            yhat_proba = list(best_param.predict_proba(X_test)[:,1])# reture the probability of predicting '1'
            y_pred.append(yhat_proba)
            y_true.append(list(y_test[0]))
            # report progress
            print('>est=%.3f, cfg=%s' % (result.best_score_, result.best_params_))
            # Calculate roc_auc on the hold out dataset
        print('auc: %.3f' % roc_auc_score(y_true, y_pred))
        print('*'*50)



In [12]:
# FOR CHINENSE READERS
rad = rads[0]
PROB = []
PRED = []
TRUE = []
for key,value in Side_View.items():
    cancer_sort = value.sort_values(by = rad)
    cancer_sort_copy = cancer_sort.copy()
    cancer_sort_copy['percentile'] = pd.qcut(cancer_sort_copy[rad], 3, labels=[0,3,1])
    cancer_sort_drop = pd.concat([cancer_sort_copy[cancer_sort_copy['percentile']==0],cancer_sort_copy[cancer_sort_copy['percentile']==1]],axis = 0)

    print('ThIS IS {} SIDE VIEW'.format(key),'\n\n')
    X = cancer_sort_drop.loc[:,"feat.RA.Normalized.1":"feat.RA.Normalized.203"].to_numpy()
    y = cancer_sort_drop['percentile'].to_numpy().reshape(-1,1)
    selector = SelectFromModel(estimator=RandomForestClassifier(),max_features = 20)
    pipe = Pipeline([ ('scaler',MinMaxScaler()),
                     ('features',selector),
                     ('rf',RandomForestClassifier(random_state=42))]) 
    param = {"features__max_features":[5,10,15],
            "rf__max_depth":[1,5,10],
             "rf__max_samples":[0.1,0.5,1.0],
            "rf__n_estimators":[50,100,1000] } 
    probability, pre_label,true_value = pipeline_evaluate(X,y,pipe, param)
    
    AUC_score = roc_auc_score(true_value, probability)
    Accuracy = accuracy_score(true_value,pre_label)
    
    cm1 = confusion_matrix(true_value,pre_label)
    sensitivity1 = cm1[0,0]/(cm1[0,0]+cm1[0,1])
    specificity1 = cm1[1,1]/(cm1[1,0]+cm1[1,1])
    print('Specificity : ', specificity1)
    print("The AUC for this view is :", AUC_score)
    print("The ACC for this view is :", Accuracy)
    print("The sen for this view is :", sensitivity1)
    print("The spe for this view is :", specificity1)
    
    PROB.extend(probability)
    PRED.append(pre_label)
    

ThIS IS Case_based_CC_CancerLesion SIDE VIEW 


>est=0.662, cfg={'features__max_features': 10, 'rf__max_depth': 10, 'rf__max_samples': 0.5, 'rf__n_estimators': 100}
>est=0.639, cfg={'features__max_features': 15, 'rf__max_depth': 1, 'rf__max_samples': 1.0, 'rf__n_estimators': 100}
>est=0.685, cfg={'features__max_features': 15, 'rf__max_depth': 1, 'rf__max_samples': 0.5, 'rf__n_estimators': 50}
>est=0.741, cfg={'features__max_features': 10, 'rf__max_depth': 5, 'rf__max_samples': 1.0, 'rf__n_estimators': 50}
>est=0.630, cfg={'features__max_features': 10, 'rf__max_depth': 1, 'rf__max_samples': 0.5, 'rf__n_estimators': 50}
>est=0.759, cfg={'features__max_features': 15, 'rf__max_depth': 5, 'rf__max_samples': 1.0, 'rf__n_estimators': 100}
>est=0.833, cfg={'features__max_features': 5, 'rf__max_depth': 1, 'rf__max_samples': 1.0, 'rf__n_estimators': 1000}
>est=0.824, cfg={'features__max_features': 10, 'rf__max_depth': 10, 'rf__max_samples': 1.0, 'rf__n_estimators': 50}
>est=0.602, cfg={'features

In [13]:
PROB = np.array(PROB).reshape(2,-1)
PROB_max = np.max(PROB,axis=0)

label = []
for i in range(len(PROB_max)):
    if PROB_max[i]>0.5:
        label.append(1)
    else:
        label.append(0)
         
# calculate sensitivity, specificity, accuracy
Accuracy = accuracy_score(true_value,label)
cm1 = confusion_matrix(true_value,label)
sensitivity1 = cm1[0,0]/(cm1[0,0]+cm1[0,1])
specificity1 = cm1[1,1]/(cm1[1,0]+cm1[1,1])

print("Accuray is :",Accuracy )
print("Sensitivity is :", sensitivity1)
print("Specificity is :", specificity1)

# calculate pipeline AUC
AUC_score = roc_auc_score(true_value, PROB_max)
print("AUC for normal pipeline for Australian readers is {:.3f}".format(AUC_score))

# get confidence interval
n_bootstraps = 2000
rng_seed = 42  # control reproducibility
bootstrapped_scores = []

rng = np.random.RandomState(rng_seed)
for i in range(n_bootstraps):
    # bootstrap by sampling with replacement on the prediction indices
    indices = rng.randint(0, len(PROB_max), len(PROB_max))
    if len(np.unique(np.array(true_value)[indices])) < 2:
        # We need at least one positive and one negative sample for ROC AUC
        # to be defined: reject the sample
        continue

    score = roc_auc_score(np.array(true_value)[indices], PROB_max[indices])
    bootstrapped_scores.append(score)
#     print("Bootstrap #{} ROC area: {:0.3f}".format(i + 1, score))

sorted_scores = np.array(bootstrapped_scores)
sorted_scores.sort()
confidence_lower = sorted_scores[int(0.025 * len(sorted_scores))]
confidence_upper = sorted_scores[int(0.975 * len(sorted_scores))]
print("Confidence interval for the score: [{:0.3f} - {:0.3}]".format(
    confidence_lower, confidence_upper))

Accuray is : 0.4
Sensitivity is : 0.0
Specificity is : 0.8571428571428571
AUC for normal pipeline for Australian readers is 0.348
Confidence interval for the score: [0.000 - 0.705]


In [16]:
# FOR Australian READERS
rad = rads[1]
PROB = []
PRED = []
TRUE = []

for key,value in Side_View.items():
    cancer_sort = value.sort_values(by = rad)
    cancer_sort_copy = cancer_sort.copy()
    cancer_sort_copy['percentile'] = pd.qcut(cancer_sort_copy[rad], 3, labels=[0,3,1])
    cancer_sort_drop = pd.concat([cancer_sort_copy[cancer_sort_copy['percentile']==0],cancer_sort_copy[cancer_sort_copy['percentile']==1]],axis = 0)

    print('ThIS IS {} SIDE VIEW'.format(key),'\n\n')
    X = cancer_sort_drop.loc[:,"feat.RA.Normalized.1":"feat.RA.Normalized.203"].to_numpy()
    y = cancer_sort_drop['percentile'].to_numpy().reshape(-1,1)
    selector = SelectFromModel(estimator=RandomForestClassifier(),max_features = 20)
    pipe = Pipeline([ ('scaler',MinMaxScaler()),
                     ('features',selector),
                     ('rf',RandomForestClassifier(random_state=42))]) 
    param = {"features__max_features":[5,10,15],
            "rf__max_depth":[1,5,10],
             "rf__max_samples":[0.1,0.5,1.0],
            "rf__n_estimators":[50,100,1000] } 
    probability, pre_label,true_value = pipeline_evaluate(X,y,pipe, param)
    
    AUC_score = roc_auc_score(true_value, probability)
    Accuracy = accuracy_score(true_value,pre_label)
    
    cm1 = confusion_matrix(true_value,pre_label)
    sensitivity1 = cm1[0,0]/(cm1[0,0]+cm1[0,1])
    specificity1 = cm1[1,1]/(cm1[1,0]+cm1[1,1])
    print('Specificity : ', specificity1)
    print("The AUC for this view is :", AUC_score)
    print("The ACC for this view is :", Accuracy)
    print("The sen for this view is :", sensitivity1)
    print("The spe for this view is :", specificity1)
    
    PROB.extend(probability)
    PRED.append(pre_label)

ThIS IS Case_based_CC_CancerLesion SIDE VIEW 


>est=0.852, cfg={'features__max_features': 5, 'rf__max_depth': 5, 'rf__max_samples': 0.5, 'rf__n_estimators': 1000}
>est=0.861, cfg={'features__max_features': 10, 'rf__max_depth': 10, 'rf__max_samples': 0.5, 'rf__n_estimators': 50}
>est=0.796, cfg={'features__max_features': 15, 'rf__max_depth': 5, 'rf__max_samples': 1.0, 'rf__n_estimators': 100}
>est=0.806, cfg={'features__max_features': 5, 'rf__max_depth': 1, 'rf__max_samples': 0.5, 'rf__n_estimators': 50}
>est=0.667, cfg={'features__max_features': 5, 'rf__max_depth': 1, 'rf__max_samples': 0.5, 'rf__n_estimators': 1000}
>est=0.657, cfg={'features__max_features': 5, 'rf__max_depth': 1, 'rf__max_samples': 0.5, 'rf__n_estimators': 50}
>est=0.713, cfg={'features__max_features': 5, 'rf__max_depth': 10, 'rf__max_samples': 0.5, 'rf__n_estimators': 50}
>est=0.759, cfg={'features__max_features': 10, 'rf__max_depth': 1, 'rf__max_samples': 0.5, 'rf__n_estimators': 100}
>est=0.704, cfg={'features__m

In [17]:
PROB = np.array(PROB).reshape(2,-1)
PROB_max = np.max(PROB,axis=0)

label = []
for i in range(len(PROB_max)):
    if PROB_max[i]>0.5:
        label.append(1)
    else:
        label.append(0)
         
# calculate sensitivity, specificity, accuracy
Accuracy = accuracy_score(true_value,label)
cm1 = confusion_matrix(true_value,label)
sensitivity1 = cm1[0,0]/(cm1[0,0]+cm1[0,1])
specificity1 = cm1[1,1]/(cm1[1,0]+cm1[1,1])

print("Accuray is :",Accuracy )
print("Sensitivity is :", sensitivity1)
print("Specificity is :", specificity1)

# calculate pipeline AUC
AUC_score = roc_auc_score(true_value, PROB_max)
print("AUC for normal pipeline for Australian readers is {:.3f}".format(AUC_score))

# get confidence interval
n_bootstraps = 2000
rng_seed = 42  # control reproducibility
bootstrapped_scores = []

rng = np.random.RandomState(rng_seed)
for i in range(n_bootstraps):
    # bootstrap by sampling with replacement on the prediction indices
    indices = rng.randint(0, len(PROB_max), len(PROB_max))
    if len(np.unique(np.array(true_value)[indices])) < 2:
        # We need at least one positive and one negative sample for ROC AUC
        # to be defined: reject the sample
        continue

    score = roc_auc_score(np.array(true_value)[indices], PROB_max[indices])
    bootstrapped_scores.append(score)
#     print("Bootstrap #{} ROC area: {:0.3f}".format(i + 1, score))

sorted_scores = np.array(bootstrapped_scores)
sorted_scores.sort()
confidence_lower = sorted_scores[int(0.025 * len(sorted_scores))]
confidence_upper = sorted_scores[int(0.975 * len(sorted_scores))]
print("Confidence interval for the score: [{:0.3f} - {:0.3}]".format(
    confidence_lower, confidence_upper))

Accuray is : 0.5
Sensitivity is : 0.5
Specificity is : 0.5
AUC for normal pipeline for Australian readers is 0.367
Confidence interval for the score: [0.103 - 0.692]


## Location Based
### Cancer Lesion Side

In [18]:
lesion_side = pd.read_csv("/Users/jessy/Desktop/笔记本/Radiomics Data Analysis/diffScore w_out 10 dep Cn/lesion_side.csv")
case_feat_location = pd.read_csv("/Users/jessy/Desktop/笔记本/Radiomics Data Analysis/diffScore w_out 10 dep Cn/case_feat_location_diffScore/RA_norm_feat.csv")
cancer_location = case_feat_location.merge(lesion_side.rename(columns={"LesionNumber":"LesionNum"}), how='left',on = ["CaseName","LesionNum"])
CC_CL_LC = cancer_location.loc[cancer_location["Side"]==cancer_location["LesionSide"],:].loc[cancer_location["View"]=="'CC'",:]
MLO_CL_LC = cancer_location.loc[cancer_location["Side"]==cancer_location["LesionSide"],:].loc[cancer_location["View"]=="'MLO'",:]

In [19]:
CC_CL_LC = CC_CL_LC[CC_CL_LC.LesionNum!=2]
MLO_CL_LC = MLO_CL_LC[MLO_CL_LC.LesionNum!=2]

In [20]:
# Cancer lesion side 
Side_View = {}
Side_View = {'Loc_based_CC_CancerLesion':CC_CL_LC,
             'Loc_based_MLO_CancerLesion': MLO_CL_LC}
rads = ['diffScore.CN','diffScore.AU'] 

# CN and AU Logistic Regression 02
for index, rad in enumerate(rads):
    for key,value in Side_View.items():
        cancer_sort = value.sort_values(by = rad)
        cancer_sort_copy = cancer_sort.copy()
        cancer_sort_copy['percentile'] = pd.cut(cancer_sort_copy[rad], Bins[index], labels=[1,3,0])
        cancer_sort_drop = pd.concat([cancer_sort_copy[cancer_sort_copy['percentile']==1],cancer_sort_copy[cancer_sort_copy['percentile']==0]],axis = 0)

        print('ThIS IS {} SIDE VIEW'.format(key),'\n\n')
        X = cancer_sort_drop.loc[:,0:202]
        y = cancer_sort_drop['percentile'].to_numpy().reshape(-1,1)



        # prepare data (log transform)
        from scipy.stats import skew
        skewness = X.apply(lambda x: skew(x))
        skewed_feats = skewness[skewness > 0.75]
        skewed_feats = skewed_feats.index
        minimum = X[skewed_feats].min(axis = 1).min()
        skewed_feats = np.log1p(X[skewed_feats]+ abs(minimum))
        X = X.to_numpy()  

        pipe_lr2 = Pipeline([
            ('scaler',MinMaxScaler()),
            ('features',combined_features),
            ('lr',LogisticRegression(solver='liblinear',random_state=42))])
        param_lr2 = {'features__univ_select__k':[5,10,20,30],
                    'lr__penalty':['l1', 'l2'],
                    'lr__C':[1000,100,10,1.0,0.1,0.01,0.001]} 
        print('*'*30)
        print('Below is the results from the LR02:')
        cv_outer = LeaveOneOut()
        y_true,y_pred = list(),list()
        for train_ix, test_ix in cv_outer.split(X):
            X_train_, X_test = X[train_ix, :], X[test_ix, :]
            y_train_, y_test = y[train_ix], y[test_ix]

            # inner loop for feature selection and hyperparameter tuning 
            cv_inner = StratifiedKFold(n_splits=3, shuffle=True, random_state=24)
            result = GridSearchCV(pipe_lr2, param_lr2, cv = cv_inner, n_jobs= -1,scoring = 'roc_auc',refit=True).fit(X_train_, y_train_.ravel())
            best_param = result.best_estimator_
            # evaluate model on the hold out validation set
            yhat_proba = list(best_param.predict_proba(X_test)[:,1])
            y_pred.append(yhat_proba)
            y_true.append(list(y_test[0]))
            # report progress
            print('>est=%.3f, cfg=%s' % (result.best_score_, result.best_params_))
           # Calculate roc_auc on the hold out dataset
        print('auc: %.3f' % roc_auc_score(y_true, y_pred))       
        print('*'*30)

# CN and AU Logistic Regression 1 3 
for index, rad in enumerate(rads):
    for key,value in Side_View.items():
        cancer_sort = value.sort_values(by = rad)
        cancer_sort_copy = cancer_sort.copy()
        cancer_sort_copy['percentile'] = pd.cut(cancer_sort_copy[rad], Bins[index], labels=[1,3,0])
        cancer_sort_drop = pd.concat([cancer_sort_copy[cancer_sort_copy['percentile']==1],cancer_sort_copy[cancer_sort_copy['percentile']==0]],axis = 0)

        print('ThIS IS {} SIDE VIEW'.format(key),'\n\n')
        X = cancer_sort_drop.loc[:,0:202]
        y = cancer_sort_drop['percentile'].to_numpy().reshape(-1,1)


        # prepare data (log transform)
        from scipy.stats import skew
        skewness = X.apply(lambda x: skew(x))
        skewed_feats = skewness[skewness > 0.75]
        skewed_feats = skewed_feats.index
        minimum = X[skewed_feats].min(axis = 1).min()
        skewed_feats = np.log1p(X[skewed_feats]+ abs(minimum))
        X = X.to_numpy()
        
        # Logistic Regression 01 (log transform, Scaler, RFECV, LR)
        pipe01 = Pipeline([('scaler',MinMaxScaler()),
                         ('features',RFECV(estimator = LogisticRegression(solver='liblinear'),cv =3, scoring ='roc_auc')),
                         ('lr',LogisticRegression(solver='liblinear',random_state=42))])
        param01 = {'lr__penalty':['l1','l2'],
                'lr__C':[1000,100,10,1.0,0.1,0.01,0.001]} 
        print('*'*30)
        print('Below is the results from the LR01:')
        self_LR_pipe(X,y,pipe01,param01)
        print('*'*30)
        
        # Logistic Regression 03 (log transform, scaler, SelectFromModel, LR)
        selector = SelectFromModel(estimator=LogisticRegression(solver = 'liblinear'))
        pipe_lr03 = Pipeline([ 
                ('scaler',MinMaxScaler()),('features',selector),
            ('lr',LogisticRegression(solver = 'liblinear',random_state=42))]) 
        param_lr03 = {'features__max_features':[20,30,40],
                        'lr__penalty':['l1', 'l2'],
                        'lr__C':[1000,100,10,1.0,0.1,0.01,0.001]} 
        print('*'*30)
        print('Below is the results from the LR03:')               
        self_LR_pipe(X,y,pipe=pipe_lr03,param=param_lr03)
        print('*'*30)

# CN and AU Random Forest01
for index, rad in enumerate(rads):
    for key,value in Side_View.items():
        cancer_sort = value.sort_values(by = rad)
        cancer_sort_copy = cancer_sort.copy()
        cancer_sort_copy['percentile'] = pd.cut(cancer_sort_copy[rad], Bins[index], labels=[1,3,0])
        cancer_sort_drop = pd.concat([cancer_sort_copy[cancer_sort_copy['percentile']==1],cancer_sort_copy[cancer_sort_copy['percentile']==0]],axis = 0)

        print('ThIS IS {} SIDE VIEW'.format(key),'\n\n')
        X = cancer_sort_drop.loc[:,0:202].to_numpy()
        y = cancer_sort_drop['percentile'].to_numpy().reshape(-1,1)


        
        # Random forest 01 (scaler, pca + mututal info classifier, RandomForest)
        pca = PCA(n_components = 0.9) 
        selection = SelectKBest(mutual_info_classif,k=10)
        combined_features = FeatureUnion([("pca", pca), ("univ_select", selection)])
        pipe = Pipeline([ ('scaler',MinMaxScaler()),
                         ('features',combined_features),
                         ('rf',RandomForestClassifier(random_state=42))]) 
        param = {'features__univ_select__k':[10,20,30,40],
                 'rf__max_features':['sqrt','log2'],
                 'rf__n_estimators':[50,100,1000,2000]} 
        print('*'*50)
        print('Below is the results from the RF01:')
        cv_outer = LeaveOneOut()
        y_true,y_pred = list(),list()
        for train_ix, test_ix in cv_outer.split(X):
            X_train, X_test = X[train_ix, :], X[test_ix, :]
            y_train, y_test = y[train_ix], y[test_ix]

            ## inner loop for feature selection and hyperparameter tuning 
            cv_inner = StratifiedKFold(n_splits=3, shuffle=True, random_state=24)
            result = GridSearchCV(pipe, param, cv = cv_inner, scoring = 'roc_auc',n_jobs = -1,refit=True).fit(X_train, y_train.ravel())
            best_param = result.best_estimator_
            # evaluate model on the hold out evaluation dataset
            yhat_proba = list(best_param.predict_proba(X_test)[:,1])# reture the probability of predicting '1'
            y_pred.append(yhat_proba)
            y_true.append(list(y_test[0]))
            # report progress
            print('>est=%.3f, cfg=%s' % (result.best_score_, result.best_params_))
            # Calculate roc_auc on the hold out dataset
        print('auc: %.3f' % roc_auc_score(y_true, y_pred))
        print('*'*50)

In [10]:
# # CN and AU Random Forest 2
# df_location = {}
# AUC = dict()
# ACU = dict()
# SEN = dict()
# SPE = dict()
# for index, rad in enumerate(rads):
#     for key,value in Side_View.items():
#         cancer_sort = value.sort_values(by = rad)
#         cancer_sort_copy = cancer_sort.copy()
#         cancer_sort_copy['percentile'] = pd.qcut(cancer_sort_copy[rad], 3, labels=[1,3,0])
#         cancer_sort_drop = pd.concat([cancer_sort_copy[cancer_sort_copy['percentile']==1],cancer_sort_copy[cancer_sort_copy['percentile']==0]],axis = 0)

#         print('ThIS IS {} SIDE VIEW'.format(key),'\n\n')
#         X = cancer_sort_drop.loc[:,"feat.RA.Normalized.1":"feat.RA.Normalized.203"].to_numpy()
#         y = cancer_sort_drop['percentile'].to_numpy().reshape(-1,1)
#         over = SMOTE(sampling_strategy="minority",random_state=3)
#         X,y = over.fit_resample(X,y)

#         # Random Forest 02 (scaler, SelectFromModel, RandomForest)
#         selector = SelectFromModel(estimator=RandomForestClassifier(),max_features = 10)
#         pipe = Pipeline([ ('scaler',MinMaxScaler()),
#                          ('features',selector),
#                          ('rf',RandomForestClassifier(random_state=9))]) 
#         param = {'features__max_features':[5,10,15],
#                          'rf__max_features':['sqrt'], 
#                      'rf__n_estimators':[50,100,1000] } 
#         print('*'*50)
#         print('Below is the results from the RF02:')
#         Dict,AUC_score,Accuracy,sensitivity1,specificity1  = self_rf_pipe(X,y,pipe,param)  
#         print('*'*50) 
#         df_location[key]=Dict
#         AUC[key]=AUC_score
#         ACU[key]=Accuracy
#         SEN[key]=sensitivity1
#         SPE[key]=specificity1

ThIS IS Loc_based_CC_CancerLesion SIDE VIEW 


**************************************************
Below is the results from the RF02:
>est=0.741, cfg={'features__max_features': 5, 'rf__max_features': 'sqrt', 'rf__n_estimators': 100}
>est=0.630, cfg={'features__max_features': 5, 'rf__max_features': 'sqrt', 'rf__n_estimators': 1000}
>est=0.806, cfg={'features__max_features': 15, 'rf__max_features': 'sqrt', 'rf__n_estimators': 1000}
>est=0.727, cfg={'features__max_features': 10, 'rf__max_features': 'sqrt', 'rf__n_estimators': 100}
>est=0.759, cfg={'features__max_features': 5, 'rf__max_features': 'sqrt', 'rf__n_estimators': 100}
>est=0.750, cfg={'features__max_features': 15, 'rf__max_features': 'sqrt', 'rf__n_estimators': 100}
>est=1.000, cfg={'features__max_features': 5, 'rf__max_features': 'sqrt', 'rf__n_estimators': 50}
>est=0.694, cfg={'features__max_features': 5, 'rf__max_features': 'sqrt', 'rf__n_estimators': 50}
>est=0.704, cfg={'features__max_features': 5, 'rf__max_features': 'sqrt

>est=0.562, cfg={'features__max_features': 5, 'rf__max_features': 'sqrt', 'rf__n_estimators': 100}
>est=0.494, cfg={'features__max_features': 5, 'rf__max_features': 'sqrt', 'rf__n_estimators': 50}
>est=0.512, cfg={'features__max_features': 5, 'rf__max_features': 'sqrt', 'rf__n_estimators': 1000}
>est=0.451, cfg={'features__max_features': 5, 'rf__max_features': 'sqrt', 'rf__n_estimators': 1000}
>est=0.472, cfg={'features__max_features': 10, 'rf__max_features': 'sqrt', 'rf__n_estimators': 100}
>est=0.549, cfg={'features__max_features': 10, 'rf__max_features': 'sqrt', 'rf__n_estimators': 1000}
0 : 3
2 : 1
4 : 2
7 : 1
8 : 1
9 : 2
10 : 1
11 : 1
17 : 1
18 : 1
19 : 1
21 : 1
22 : 1
32 : 1
41 : 1
45 : 1
48 : 2
51 : 1
62 : 1
68 : 2
69 : 6
70 : 4
71 : 3
72 : 1
73 : 5
74 : 1
75 : 2
82 : 1
97 : 1
100 : 1
101 : 1
109 : 12
112 : 1
113 : 2
114 : 5
119 : 1
121 : 2
122 : 4
124 : 1
125 : 3
126 : 5
134 : 1
135 : 2
138 : 1
142 : 4
148 : 1
160 : 2
165 : 1
171 : 2
178 : 1
179 : 1
180 : 2
182 : 1
184 : 1
187 

In [22]:
# FOR CHINENSE READERS
rad = rads[0]
PROB = []
PRED = []
TRUE = []
for key,value in Side_View.items():
    cancer_sort = value.sort_values(by = rad)
    cancer_sort_copy = cancer_sort.copy()
    cancer_sort_copy['percentile'] = pd.qcut(cancer_sort_copy[rad], 3, labels=[0,3,1])
    cancer_sort_drop = pd.concat([cancer_sort_copy[cancer_sort_copy['percentile']==1],cancer_sort_copy[cancer_sort_copy['percentile']==0]],axis = 0)

    print('ThIS IS {} SIDE VIEW'.format(key),'\n\n')
    X = cancer_sort_drop.loc[:,"feat.RA.Normalized.1":"feat.RA.Normalized.203"].to_numpy()
    y = cancer_sort_drop['percentile'].to_numpy().reshape(-1,1)
    selector = SelectFromModel(estimator=RandomForestClassifier(),max_features = 20)
    pipe = Pipeline([ ('scaler',MinMaxScaler()),
                     ('features',selector),
                     ('rf',RandomForestClassifier(random_state=42))]) 
    param = {"features__max_features":[5,10,15],
            "rf__max_depth":[1,5,10],
             "rf__max_samples":[0.1,0.5,1.0],
            "rf__n_estimators":[50,100,1000] } 
    probability, pre_label,true_value = pipeline_evaluate(X,y,pipe, param)
    
    AUC_score = roc_auc_score(true_value, probability)
    Accuracy = accuracy_score(true_value,pre_label)
    
    cm1 = confusion_matrix(true_value,pre_label)
    sensitivity1 = cm1[0,0]/(cm1[0,0]+cm1[0,1])
    specificity1 = cm1[1,1]/(cm1[1,0]+cm1[1,1])
    print('Specificity : ', specificity1)
    print("The AUC for this view is :", AUC_score)
    print("The ACC for this view is :", Accuracy)
    print("The sen for this view is :", sensitivity1)
    print("The spe for this view is :", specificity1)
    
    PROB.extend(probability)
    PRED.append(pre_label)

ThIS IS Loc_based_CC_CancerLesion SIDE VIEW 


>est=0.773, cfg={'features__max_features': 5, 'rf__max_depth': 10, 'rf__max_samples': 1.0, 'rf__n_estimators': 50}
>est=0.750, cfg={'features__max_features': 5, 'rf__max_depth': 1, 'rf__max_samples': 1.0, 'rf__n_estimators': 50}
>est=0.681, cfg={'features__max_features': 5, 'rf__max_depth': 1, 'rf__max_samples': 1.0, 'rf__n_estimators': 50}
>est=0.750, cfg={'features__max_features': 5, 'rf__max_depth': 10, 'rf__max_samples': 1.0, 'rf__n_estimators': 1000}
>est=0.704, cfg={'features__max_features': 5, 'rf__max_depth': 1, 'rf__max_samples': 1.0, 'rf__n_estimators': 1000}
>est=0.759, cfg={'features__max_features': 5, 'rf__max_depth': 1, 'rf__max_samples': 1.0, 'rf__n_estimators': 50}
>est=0.671, cfg={'features__max_features': 5, 'rf__max_depth': 10, 'rf__max_samples': 0.5, 'rf__n_estimators': 100}
>est=0.731, cfg={'features__max_features': 10, 'rf__max_depth': 1, 'rf__max_samples': 1.0, 'rf__n_estimators': 100}
>est=0.759, cfg={'features__max

In [23]:
PROB = np.array(PROB).reshape(2,-1)
# get the maximum prob among predictions of the four views
PROB_max = np.max(PROB,axis=0)

label = []
for i in range(len(PROB_max)):
    if PROB_max[i]>0.5:
        label.append(1)
    else:
        label.append(0)
         
# calculate sensitivity, specificity, accuracy
Accuracy = accuracy_score(true_value,label)
cm1 = confusion_matrix(true_value,label)
sensitivity1 = cm1[0,0]/(cm1[0,0]+cm1[0,1])
specificity1 = cm1[1,1]/(cm1[1,0]+cm1[1,1])
print("Accuray is :",Accuracy )
print("Sensitivity is :", sensitivity1)
print("Specificity is :", specificity1)

# calculate pipeline AUC
AUC_score = roc_auc_score(true_value, PROB_max)
print("AUC for normal pipeline for Chinese readers is {:.3f}".format(AUC_score))

# get confidence interval
n_bootstraps = 2000
rng_seed = 42  # control reproducibility
bootstrapped_scores = []

rng = np.random.RandomState(rng_seed)
for i in range(n_bootstraps):
    # bootstrap by sampling with replacement on the prediction indices
    indices = rng.randint(0, len(PROB_max), len(PROB_max))
    if len(np.unique(np.array(true_value)[indices])) < 2:
        # We need at least one positive and one negative sample for ROC AUC
        # to be defined: reject the sample
        continue

    score = roc_auc_score(np.array(true_value)[indices], PROB_max[indices])
    bootstrapped_scores.append(score)
#     print("Bootstrap #{} ROC area: {:0.3f}".format(i + 1, score))

sorted_scores = np.array(bootstrapped_scores)
sorted_scores.sort()
confidence_lower = sorted_scores[int(0.025 * len(sorted_scores))]
confidence_upper = sorted_scores[int(0.975 * len(sorted_scores))]
print("Confidence interval for the score: [{:0.3f} - {:0.3}]".format(
    confidence_lower, confidence_upper))

Accuray is : 0.7857142857142857
Sensitivity is : 0.8571428571428571
Specificity is : 0.7142857142857143
AUC for normal pipeline for Chinese readers is 0.735
Confidence interval for the score: [0.400 - 1.0]


In [25]:
# FOR AU READERS
rad = rads[1]
PROB = []
PRED = []
TRUE = []
for key,value in Side_View.items():
    cancer_sort = value.sort_values(by = rad)
    cancer_sort_copy = cancer_sort.copy()
    cancer_sort_copy['percentile'] = pd.qcut(cancer_sort_copy[rad], 3, labels=[0,3,1])
    cancer_sort_drop = pd.concat([cancer_sort_copy[cancer_sort_copy['percentile']==1],cancer_sort_copy[cancer_sort_copy['percentile']==0]],axis = 0)

    print('ThIS IS {} SIDE VIEW'.format(key),'\n\n')
    X = cancer_sort_drop.loc[:,"feat.RA.Normalized.1":"feat.RA.Normalized.203"].to_numpy()
    y = cancer_sort_drop['percentile'].to_numpy().reshape(-1,1)
    selector = SelectFromModel(estimator=RandomForestClassifier(),max_features = 20)
    pipe = Pipeline([ ('scaler',MinMaxScaler()),
                     ('features',selector),
                     ('rf',RandomForestClassifier(random_state=42))]) 
    param = {"features__max_features":[5,10,15],
            "rf__max_depth":[1,5,10],
             "rf__max_samples":[0.1,0.5,1.0],
            "rf__n_estimators":[50,100,1000] } 
    probability, pre_label,true_value = pipeline_evaluate(X,y,pipe, param)
    
    AUC_score = roc_auc_score(true_value, probability)
    Accuracy = accuracy_score(true_value,pre_label)
    
    cm1 = confusion_matrix(true_value,pre_label)
    sensitivity1 = cm1[0,0]/(cm1[0,0]+cm1[0,1])
    specificity1 = cm1[1,1]/(cm1[1,0]+cm1[1,1])
    print('Specificity : ', specificity1)
    print("The AUC for this view is :", AUC_score)
    print("The ACC for this view is :", Accuracy)
    print("The sen for this view is :", sensitivity1)
    print("The spe for this view is :", specificity1)
    
    PROB.extend(probability)
#     PRED.append(pre_label)
    

ThIS IS Loc_based_CC_CancerLesion SIDE VIEW 


>est=0.509, cfg={'features__max_features': 5, 'rf__max_depth': 5, 'rf__max_samples': 0.5, 'rf__n_estimators': 50}
>est=0.565, cfg={'features__max_features': 10, 'rf__max_depth': 5, 'rf__max_samples': 1.0, 'rf__n_estimators': 100}
>est=0.620, cfg={'features__max_features': 5, 'rf__max_depth': 1, 'rf__max_samples': 1.0, 'rf__n_estimators': 100}
>est=0.722, cfg={'features__max_features': 5, 'rf__max_depth': 10, 'rf__max_samples': 1.0, 'rf__n_estimators': 1000}
>est=0.667, cfg={'features__max_features': 15, 'rf__max_depth': 10, 'rf__max_samples': 0.5, 'rf__n_estimators': 100}
>est=0.713, cfg={'features__max_features': 5, 'rf__max_depth': 5, 'rf__max_samples': 1.0, 'rf__n_estimators': 100}
>est=0.611, cfg={'features__max_features': 10, 'rf__max_depth': 10, 'rf__max_samples': 0.5, 'rf__n_estimators': 100}
>est=0.500, cfg={'features__max_features': 5, 'rf__max_depth': 1, 'rf__max_samples': 0.1, 'rf__n_estimators': 50}
>est=0.528, cfg={'features__

In [26]:
PROB = np.array(PROB).reshape(2,-1)
# get the maximum prob among predictions of the four views
PROB_max = np.max(PROB,axis=0)

label = []
for i in range(len(PROB_max)):
    if PROB_max[i]>0.5:
        label.append(1)
    else:
        label.append(0)
         
# calculate sensitivity, specificity, accuracy
Accuracy = accuracy_score(true_value,label)
cm1 = confusion_matrix(true_value,label)
sensitivity1 = cm1[0,0]/(cm1[0,0]+cm1[0,1])
specificity1 = cm1[1,1]/(cm1[1,0]+cm1[1,1])
print("Accuray is :",Accuracy )
print("Sensitivity is :", sensitivity1)
print("Specificity is :", specificity1)

# calculate pipeline AUC
AUC_score = roc_auc_score(true_value, PROB_max)
print("AUC for normal pipeline for Australian readers is {:.3f}".format(AUC_score))

# get confidence interval
n_bootstraps = 2000
rng_seed = 42  # control reproducibility
bootstrapped_scores = []

rng = np.random.RandomState(rng_seed)
for i in range(n_bootstraps):
    # bootstrap by sampling with replacement on the prediction indices
    indices = rng.randint(0, len(PROB_max), len(PROB_max))
    if len(np.unique(np.array(true_value)[indices])) < 2:
        # We need at least one positive and one negative sample for ROC AUC
        # to be defined: reject the sample
        continue

    score = roc_auc_score(np.array(true_value)[indices], PROB_max[indices])
    bootstrapped_scores.append(score)
#     print("Bootstrap #{} ROC area: {:0.3f}".format(i + 1, score))

sorted_scores = np.array(bootstrapped_scores)
sorted_scores.sort()
confidence_lower = sorted_scores[int(0.025 * len(sorted_scores))]
confidence_upper = sorted_scores[int(0.975 * len(sorted_scores))]
print("Confidence interval for the score: [{:0.3f} - {:0.3}]".format(
    confidence_lower, confidence_upper))

Accuray is : 0.4666666666666667
Sensitivity is : 0.5555555555555556
Specificity is : 0.3333333333333333
AUC for normal pipeline for Australian readers is 0.481
Confidence interval for the score: [0.160 - 0.804]
